In [61]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import scipy.stats as ss

def nagelkerke_r2(model):
    # Arg: model = smf.logit(...).fit()
    # See: https://stats.stackexchange.com/questions/183699/how-to-calculate-pseudo-r2-when-using-logistic-regression-on-aggregated-data-fil
    ll0 = model.llnull
    ll = model.llf
    n = model.nobs
    R2cs = 1 - np.exp((2 * (ll0 - ll))/n)
    r2ML_max = 1 - np.exp(ll0 * 2/n)
    R2nag = R2cs/r2ML_max
    return R2nag

def likelihood_ratio_test(full_model, reduced_model):
    full_ll = full_model.llf
    reduced_ll = reduced_model.llf
    LR_statistic = -2*(reduced_ll-full_ll)
    p_val = ss.chi2.sf(LR_statistic, 2)
    return LR_statistic, p_val

In [87]:
pheno_name = "BIP" # SCZ:50000 DEP:50000 AUT:100000 BIP:100000
n_snp = 100000
pheno_pgs_fname_dict = {"BIP":"BIP3", "SCZ":"SCZ", "DEP":"MDD-mvp-pgc", "AUT":"ASD"}
fname_pheno = "/cluster/projects/p33/users/alexeas/elleke/pleioprs/pheno/pheno.txt"
fname_covar = "/cluster/projects/p33/users/alexeas/elleke/pleioprs/pheno/covar.txt"
fname_fdr = f"/cluster/projects/p33/users/alexeas/elleke/pleioprs/prs/{pheno_pgs_fname_dict[pheno_name]}.FDR.{n_snp}.best"
fname_pval = f"/cluster/projects/p33/users/alexeas/elleke/pleioprs/prs/{pheno_pgs_fname_dict[pheno_name]}.PVAL.{n_snp}.best"
pheno = pd.read_csv(fname_pheno, sep='\t')
covar = pd.read_csv(fname_covar, sep='\t', usecols="IID SEX AGE PC1 PC2 PC3 PC4 PC5 PC6 PC7 PC8 PC9 PC10".split())
valid_pheno_i = pheno[pheno_name].notna()
pheno = pheno.loc[valid_pheno_i,:]
print(f"{valid_pheno_i.sum()} valid phenotypes for {pheno_name}")
pgs_fdr = pd.read_csv(fname_fdr, sep=' ', usecols=["IID","PRS"])
pgs_fdr.rename(columns={"PRS":"PRS_FDR"}, inplace=True)
pgs_pval = pd.read_csv(fname_pval, sep=' ', usecols=["IID","PRS"])
pgs_pval.rename(columns={"PRS":"PRS_PVAL"}, inplace=True)
pheno = pheno.merge(pgs_fdr, on="IID", how='inner')
pheno = pheno.merge(pgs_pval, on="IID", how='inner')
pheno = pheno.merge(covar, on="IID", how='inner')
pheno[pheno_name] -= 1 # convert controls to 0, cases to 1
pheno[pheno_name] = pheno[pheno_name].astype(int)
print(f"{pheno.shape[0]} valid individuals.")
pheno.head(3)

1536 valid phenotypes for BIP
1536 valid individuals.


,FID,IID,SCZ,BIP,AUT,DEP,PRS_FDR,PRS_PVAL,SEX,AGE,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10
0,PACCOLA_A1024,PACCOLA_A1024,NaN,1,NaN,NaN,-0.001332,-0.001643,0,19.0,-0.000020,-0.002587,-0.002234,-0.001363,0.004238,-0.002274,-0.001139,0.004040,0.000862,-0.001595
1,PACIPQP_A1026,PACIPQP_A1026,NaN,1,NaN,NaN,-0.001321,-0.001650,0,21.0,0.003596,-0.001970,0.000149,0.000439,0.001253,-0.000244,-0.001252,-0.004339,0.002556,0.002929
2,PACIPRF_A1037,PACIPRF_A1037,NaN,1,NaN,NaN,-0.001330,-0.001645,0,35.0,0.002817,-0.001537,0.000862,0.000476,0.004397,0.002781,-0.001333,-0.005446,-0.001468,0.004881


In [88]:
# Standardize PRS
if True:
    for col in ["PRS_FDR", "PRS_PVAL"]:
        pheno[col] = (pheno[col] - pheno[col].mean())/pheno[col].std()
    
    # Standardize covariates
    covariate_columns = "AGE SEX PC1 PC2 PC3 PC4 PC5 PC6 PC7 PC8 PC9 PC10".split()
    for col in covariate_columns:
        pheno[col] = (pheno[col] - pheno[col].mean())/pheno[col].std()
    covariates = ' + '.join(covariate_columns)

In [89]:
for col in ["PRS_FDR", "PRS_PVAL"]:
    print(f"{col}: mean = {pheno[col].mean()}; std = {pheno[col].std()}")
pheno[["PRS_FDR", "PRS_PVAL"]].corr()

PRS_FDR: mean = -2.7292982688701767e-16; std = 1.0
PRS_PVAL: mean = 5.773159728050814e-15; std = 1.0


,PRS_FDR,PRS_PVAL
PRS_FDR,1.000000,0.918989
PRS_PVAL,0.918989,1.000000


In [90]:
model_null = smf.logit(formula=f"{pheno_name} ~ {covariates}", data=pheno).fit()
prs_fdr = "PRS_FDR"
prs_pval = "PRS_PVAL"
model_fdr = smf.logit(formula=f"{pheno_name} ~ {prs_fdr} + {covariates}", data=pheno).fit()
model_pval = smf.logit(formula=f"{pheno_name} ~ {prs_pval} + {covariates}", data=pheno).fit()
model_both = smf.logit(formula=f"{pheno_name} ~ {prs_fdr} + {prs_pval} + {covariates}", data=pheno).fit()

Optimization terminated successfully.
         Current function value: 0.584375
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.562464
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.566781
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.562435
         Iterations 5


In [91]:
print(f"Nagelkerke R2 = {nagelkerke_r2(model_both) - nagelkerke_r2(model_null)}")
model_both.summary2()

Nagelkerke R2 = 0.0575327136162659


<class 'statsmodels.iolib.summary2.Summary'>
"""
                         Results: Logit
=================================================================
Model:              Logit            Method:           MLE       
Dependent Variable: BIP              Pseudo R-squared: 0.081     
Date:               2023-12-06 01:19 AIC:              1757.8008 
No. Observations:   1536             BIC:              1837.8549 
Df Model:           14               Log-Likelihood:   -863.90   
Df Residuals:       1521             LL-Null:          -940.14   
Converged:          1.0000           LLR p-value:      2.2843e-25
No. Iterations:     5.0000           Scale:            1.0000    
------------------------------------------------------------------
              Coef.   Std.Err.     z      P>|z|    [0.025   0.975]
------------------------------------------------------------------
Intercept    -0.9291    0.0602  -15.4214  0.0000  -1.0472  -0.8110
PRS_FDR       0.5434    0.1497    3.6291  0.0003   0.2499   0.8369
PRS_PVAL     -0.0448    0.1493   -0.2997  0.7644  -0.3374   0.2479
AGE           0.2103    0.0582    3.6160  0.0003   0.0963   0.3242
SEX          -0.2781    0.0594   -4.6819  0.0000  -0.3945  -0.1617
PC1           0.0734    0.0626    1.1723  0.2411  -0.0493   0.1962
PC2           0.1698    0.0794    2.1389  0.0324   0.0142   0.3254
PC3          -0.3068    0.0847   -3.6198  0.0003  -0.4729  -0.1407
PC4           0.1449    0.0778    1.8627  0.0625  -0.0076   0.2974
PC5           0.0717    0.0667    1.0747  0.2825  -0.0591   0.2024
PC6           0.1708    0.0622    2.7480  0.0060   0.0490   0.2927
PC7          -0.0359    0.0616   -0.5832  0.5597  -0.1566   0.0848
PC8          -0.0763    0.0606   -1.2584  0.2083  -0.1951   0.0425
PC9          -0.0436    0.0633   -0.6895  0.4905  -0.1676   0.0804
PC10          0.0288    0.0598    0.4813  0.6303  -0.0884   0.1459
=================================================================

"""

In [92]:
delta_AIC = model_pval.aic - model_fdr.aic 
delta_AIC

13.261582813747737

In [93]:
outf = f"{pheno_name}.{n_snp}.pgs_comparison.txt"
with open(outf, 'w') as ff:
    llr_stat, llr_pval = likelihood_ratio_test(model_both, model_pval)
    tval_pval, pval_pval = model_both.tvalues["PRS_PVAL"], model_both.pvalues["PRS_PVAL"]
    tval_fdr, pval_fdr = model_both.tvalues["PRS_FDR"], model_both.pvalues["PRS_FDR"]
    ff.write(f"Likelihood ratio test statistics = {llr_stat} (p-value = {llr_pval})\n")
    ff.write(f"PRS_PVAL t-statistics = {tval_pval} (p-value = {pval_pval})\n")
    ff.write(f"PRS_FDR t-statistics = {tval_fdr} (p-value = {pval_fdr})\n")